In [16]:
import os
import json
import pandas as pd 
import traceback

In [17]:
from langchain.chat_models import ChatOpenAI

In [18]:
from dotenv import load_dotenv

load_dotenv() 

True

In [19]:
KEY=os.getenv("OPENAI_API_KEY")

In [22]:
llm= ChatOpenAI(openai_api_key=KEY,model_name= "gpt-3.5-turbo",temperature=0.3)

In [21]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [14]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [15]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [16]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [17]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [18]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [19]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [20]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [21]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [22]:
file_path= r"D:\PACE_UNI\pace_uni\data.txt"

In [23]:
file_path

'D:\\PACE_UNI\\pace_uni\\data.txt'

In [24]:
with open(file_path,'r') as text:
    TEXT = text.read()

In [27]:
print(TEXT)

Machine Learning (ML) represents a transformative paradigm within the broader field of artificial intelligence (AI), revolutionizing the way computers learn and make decisions. At its core, ML is an interdisciplinary domain that merges computer science, statistics, and domain-specific expertise to enable systems to learn patterns from data and make predictions or decisions without explicit programming.

The fundamental concept behind ML is the extraction of meaningful insights from data through the identification of patterns, correlations, and trends. This process involves training algorithms on vast datasets, allowing the system to discern patterns and relationships, and subsequently apply this acquired knowledge to new, unseen data.

ML algorithms can be broadly categorized into three types: supervised learning, unsupervised learning, and reinforcement learning. In supervised learning, models are trained on labeled datasets where the algorithm learns to map input data to correspondin

In [30]:
NUMBER=5 
SUBJECT="Machine Learning"
TONE="simple"

In [28]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [31]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

c:\Users\Admin\MCQ_generator-Using-Langchain-and-OpenAI\llmenv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine Learning (ML) represents a transformative paradigm within the broader field of artificial intelligence (AI), revolutionizing the way computers learn and make decisions. At its core, ML is an interdisciplinary domain that merges computer science, statistics, and domain-specific expertise to enable systems to learn patterns from data and make predictions or decisions without explicit programming.

The fundamental concept behind ML is the extraction of meaningful insights from data through the identification of patterns, correlations, and trends. This process involves training algorithms on vast datasets, allowing the system to discern patterns and relationships, and subsequently apply this acquired knowledge to new, unseen data.

ML algorithms can be broadly categorized into three types: supervised learning, unsupervised learning, and reinforcement learning. In supervised l

In [33]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost(in doller):{cb.total_cost}")

Total Tokens:2294
Prompt Tokens:1616
Completion Tokens:678
Total Cost(in doller):0.0037800000000000004


In [53]:
response

{'text': 'Machine Learning (ML) represents a transformative paradigm within the broader field of artificial intelligence (AI), revolutionizing the way computers learn and make decisions. At its core, ML is an interdisciplinary domain that merges computer science, statistics, and domain-specific expertise to enable systems to learn patterns from data and make predictions or decisions without explicit programming.\n\nThe fundamental concept behind ML is the extraction of meaningful insights from data through the identification of patterns, correlations, and trends. This process involves training algorithms on vast datasets, allowing the system to discern patterns and relationships, and subsequently apply this acquired knowledge to new, unseen data.\n\nML algorithms can be broadly categorized into three types: supervised learning, unsupervised learning, and reinforcement learning. In supervised learning, models are trained on labeled datasets where the algorithm learns to map input data t

In [54]:
quiz = response.get("quiz")

In [58]:
quiz= json.loads(quiz)

In [60]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [62]:
quiz=pd.DataFrame(quiz_table_data)

In [64]:
quiz.to_csv("machinelearning.csv",index=False)

In [6]:
import logging
import os 
import datetime

LOG_FILE = f"{datetime.now().strftime("%m-%d-%Y %H:%M:%S")}.log"

SyntaxError: invalid syntax (3679989900.py, line 5)

In [4]:
from datetime import datetime

In [5]:
datetime.now().strftime("%m-%d-%Y %H:%M:%S")

'02-05-2024 19:43:36'

In [2]:
file_path= r"C:\Users\Admin\MCQ_generator_Using_Langchain_and_OpenAI\Response.json"

In [3]:
file_path

'C:\\Users\\Admin\\MCQ_generator_Using_Langchain_and_OpenAI\\Response.json'

In [14]:
import json
#load Json file
with open(file_path,'r') as file:
    # file = file.read()
    RESPONSE_JSON= json.load(file)

In [15]:
RESPONSE_JSON

{'1': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [12]:
print(file)

RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [ ]:
import os
# from src.mcqgenerator.constant import OUTPUT_FILE_NAME, OUTPUT_FOlder_NAME|

: 

In [ ]:
os.getcwd()

: 

In [ ]:
from src.mcqgenerator.constant import OUTPUT_FILE_NAME,OUTPUT_FOlder_NAME

: 

In [ ]:
from src.mcqgenerator.logger import logging

: 

In [ ]:
file_path = os.getcwd()

: 

In [ ]:
os.mkdir(file_path)

: 

In [ ]:
file = os.path.join(file_path,OUTPUT_FOlder_NAME)

: 

: 